**Exercise 1.9. (Stochastic volatility, random interest rate).** Consider a binomial pricing model, but at each time $n \ge 1$, the "up factor" $u_n(\omega_1 \omega_2 \ldots \omega_n)$, the "down factor" $d_n(\omega_1 \omega_2 \ldots \omega_n)$, and the interest rate $r_n(\omega_1 \omega_2 \ldots \omega_n)$ are allowed to depend on $n$ and on the first $n$ coin tosses $\omega_1 \omega_2 \ldots \omega_n$. The initial up factor $u_0$, the initial down factor $d_0$, and the initial interest rate $r_0$ are not random. More specifically, the stock price at time one is given by

$$
S_1 = \begin{cases}
u_0 S_0 \; if \; \omega_1 = H, \\
d_0 S_0 \; if \; \omega_1 = T,
\end{cases}
$$

and, for $n \ge 1$, the stock price at time $n + 1$ is given by

$$
S_{n + 1}(\omega_1 \omega_2 \ldots \omega_n \omega_{n + 1}) = \begin{cases}
u_n(\omega_1 \omega_2 \ldots \omega_n) S_n(\omega_1 \omega_2 \ldots \omega_n) \; if \; \omega_{n + 1} = H, \\
d_n(\omega_1 \omega_2 \ldots \omega_n) S_n(\omega_1 \omega_2 \ldots \omega_n) \; if \; \omega_{n + 1} = T.
\end{cases}
$$

One dollar invested in or borrowed from the money market at time zero grows to an investment or debt of $1 + r_0$ at time one, and, for $n \ge 1$, one dollar invested in or borrowed from the money market at time $n$ grows to an investment or debt of $1 + r_n(\omega_1 \omega_2 \ldots \omega_n)$ at time $n + 1$. We assume that for each $n$ and for all $\omega_1 \omega_2 \ldots \omega_n$, the no-arbitrage condition

$$0 < d_n(\omega_1 \omega_2 \ldots \omega_n) < 1 + r_n(\omega_1 \omega_2 \ldots \omega_n) < u_n(\omega_1 \omega_2 \ldots \omega_n)$$

holds. We also assume that $0 < d_0 < 1 + r_0 < u_0$.

(i) Let $N$ be a positive integer. In the model just described, provide an algorithm for determining the price at time zero for a derivative security that at time $N$ pays off a random amount $V_N$ depending on the result of the first $N$ coin tosses.

---

Looking at the proof of Threorem 1.2.2, we see that it is equally true whether $u$, $d$, and $r$ are constant or not.

We define the Toss enum:

In [1]:
@enum Toss H T

Given

$$
\begin{cases}
u(\omega_1 \omega_2 \ldots \omega_i) = u_i \\
d(\omega_1 \omega_2 \ldots \omega_i) = d_i \\
r(\omega_1 \omega_2 \ldots \omega_i) = r_i
\end{cases}
$$

and the adapted equations from formulas (1.2.15)

$$\tilde p _i = \frac{1 + r_i - d_i}{u_i - d_i}$$

and

$$\tilde q _i = \frac{u_i - 1 - r_i}{u_i - d_i},$$

we define $\tilde p (\omega_1 \omega_2 \ldots \omega_i) = \tilde p _i$:

In [2]:
p̃(ω::Toss...) = (1 + r(ω...) - d(ω...))/(u(ω...) - d(ω...));

and $\tilde q (\omega_1 \omega_2 \ldots \omega_i) = \tilde q _i$:

In [3]:
q̃(ω::Toss...) = (u(ω...) - 1 - r(ω...))/(u(ω...) - d(ω...));

Given $S_0$, we define $S(\omega_1 \omega_2 \ldots \omega_i) = S_i$:

In [4]:
function S(ω::Toss...)
    i = length(ω)
    if i == 0
        return S₀
    else
        Sᵢ₋₁ = S(ω[1 : end - 1]...)
        uᵢ₋₁ = u(ω[1 : end - 1]...)
        dᵢ₋₁ = d(ω[1 : end - 1]...)
        if ω[i] == H
            return uᵢ₋₁*Sᵢ₋₁
        else
            return dᵢ₋₁*Sᵢ₋₁
        end
    end
end;

Given the derivative payoff at time $\tau$ 

$$P(\omega_1 \omega_2 \ldots \omega_\tau) = V_\tau$$

and the adapted formula (1.2.16)

$$V_i = \frac{1}{1 + r_i} (\tilde p _i V_{i + 1}(H) + \tilde q _i V_{i + 1}(T)),$$

we define $V(\omega_1 \omega_2 \ldots \omega_i) = V_i$:

In [5]:
function V(ω::Toss...)
    i = length(ω)
    if i == τ
        # derivative payoff
        return P(ω...)
    elseif i < τ
        # formula (1.2.16)
        return 1/(1 + r(ω...))*(p̃(ω...)*V(ω..., H) + q̃(ω...)*V(ω..., T))
    else
        error("out of domain parameter")
    end
end;

---

(ii) Provide a formula for the number of shares of stock that should be held at each time $n$ ($0 \le n \le N − 1$) by a portfolio that replicates the derivatives security $V_N$.

---

From the formula (1.2.17)

$$\Delta_i = \frac{V_{i + 1}(H) - V_{i + 1}(T)}{S_{i + 1}(H) - S_{i + 1}(T)},$$

we define $\Delta(\omega_1 \omega_2 \ldots \omega_i) = \Delta_i$:

In [6]:
Δ(ω::Toss...) = (V(ω..., H) - V(ω..., T))/(S(ω..., H) - S(ω..., T));

---

(iii) Suppose the initial stock price is $S_0 = 80$, with each head the stock price increases by $10$, and with each tail the stock price decreases by $10$. In other words, $S_1(H) = 90$, $S_1(T) = 70$, $S_2(HH) = 100$, etc. Assume the interest rate is always zero. Consider a European call with strike price $80$, expiring at time five. What is the price of this call at time zero?

---

We define $S_0$:

In [7]:
S₀ = 80

80

We want

$$
S_{n + 1}(\omega_{n + 1}) = \begin{cases}
u_n S_n = S_n + 10\; if \; \omega_{n + 1} = H \\
d_n S_n = S_n - 10\; if \; \omega_{n + 1} = T
\end{cases}.
$$

Solving for $u_n$ and $d_n$, we get

$$
\begin{cases}
u_n = 1 + \frac{10}{S_n}\; if \; \omega_{n + 1} = H \\
d_n = 1 - \frac{10}{S_n}\; if \; \omega_{n + 1} = T
\end{cases}.
$$

Moreover

$$r_n(\omega_1 \omega_2 \ldots \omega_n) = 0.$$

So, we define

$$
\begin{cases}
u(\omega_1 \omega_2 \ldots \omega_i) = u_i \\
d(\omega_1 \omega_2 \ldots \omega_i) = d_i \\
r(\omega_1 \omega_2 \ldots \omega_i) = r_i
\end{cases}
$$

In [8]:
u(ω...) = 1 + 10/S(ω...)
d(ω...) = 1 - 10/S(ω...)
r(ω...) = 0
print("S₁(H) = $(S(H)), S₁(T) = $(S(T)), S₂(HH) = $(S(H, H))")

S₁(H) = 90.0, S₁(T) = 70.0, S₂(HH) = 100.0

Given the strike price $K = 80$, we define de payoff $P(\omega_1 \omega_2 \ldots \omega_\tau) = V_\tau$ of the European call:

In [9]:
K = 80
P(ω...) = max(0, S(ω...) - K);

Defining the expiring time $\tau = 5$:

In [10]:
τ = 5

5

The price of this call at time zero is: 

In [11]:
V₀ = V()

9.375000000000002

---

As we have seen in Section 1.3, some algorithms can be organized in more efficient manners.


If we define

$$
\eta(\omega) = \begin{cases}
10 \; if \; \omega = H \\
-10 \; if \; \omega = T
\end{cases},
$$

it is easy to see that the following expression is true:

$$S_i(\omega_1 \omega_2 \ldots \omega_i) = S_0 + \sum_{n = 1}^i \eta(\omega_i).$$

The sum of real numbers is commutative. Therefore we notice that the value of $S_i(\omega_1 \omega_2 \ldots \omega_i)$ depends only on the quantity of $H$ and $T$ in the sequence $\omega_1 \omega_2 \ldots \omega_i$ (e.g: $S_2(HT) = S_2(TH)$, $S_3(TTH) = S_3(THT) = S_3(HTT)$ etc).

Moreover, since the European call payoff $V_\tau(\omega_1 \omega_2 \ldots \omega_\tau)$ at time $\tau$

$$V_\tau(\omega_1 \omega_2 \ldots \omega_\tau) = (S_\tau(\omega_1 \omega_2 \ldots \omega_\tau) - K)^+$$

depends only on $S_\tau(\omega_1 \omega_2 \ldots \omega_\tau)$, we infer that it also depends only on the quantity of $H$ and $T$ in the sequence $\omega_1 \omega_2 \ldots \omega_\tau$.

Using the same logic, given that $\tilde p _i(\omega_1 \omega_2 \ldots \omega_i)$ and $\tilde q _i(\omega_1 \omega_2 \ldots \omega_i)$ depend only on $S_i(\omega_1 \omega_2 \ldots \omega_i)$ and that $r_i(\omega_1 \omega_2 \ldots \omega_i) = 0$ is constant, the European call price $V_i(\omega_1 \omega_2 \ldots \omega_i)$ at time $i$, given by the formula (1.2.16)

$$V_i(\omega_1 \omega_2 \ldots \omega_i) = \frac{1}{1 + r_i(\omega_1 \omega_2 \ldots \omega_i)} (\tilde p _i(\omega_1 \omega_2 \ldots \omega_i) V_{i + 1}(\omega_1 \omega_2 \ldots \omega_i H) + \tilde q _i(\omega_1 \omega_2 \ldots \omega_i) V_{i + 1}(\omega_1 \omega_2 \ldots \omega_i T)),$$

depends only on the quantity of $H$ and $T$ in the sequence $\omega_1 \omega_2 \ldots \omega_i$.

We can improve our implementation taking this into consideration as follows:

We define the function $\phi(\omega_1 \omega_2 \ldots \omega_i)$ that returns the quantity of $T$ and $H$ in a sequence $\omega_1 \omega_2 \ldots \omega_i$:

In [12]:
function ϕ(ω::Toss...)
    h = t = 0
    for ωᵢ ∈ ω
        if ωᵢ == H
            h += 1
        else
            t += 1
        end
    end
    return (H = h, T = t)
end;

We can [memoize](https://en.wikipedia.org/wiki/Memoization) in a dictionary `D` the results of $V_i(\omega_1 \omega_2 \ldots \omega_i)$ by the number of $H$ and $T$ in the sequence $\omega_1 \omega_2 \ldots \omega_i$.

So we define $\upsilon(\omega_1 \omega_2 \ldots \omega_i) = V_i$:

In [13]:
D = Dict()
function υ(ω::Toss...)
    return get!(D, ϕ(ω...)) do
        i = length(ω)
        if i == τ
            # derivative payoff
            return P(ω...)
        elseif i < τ
            # formula (1.2.16)
            return 1/(1 + r(ω...))*(p̃(ω...)*υ(ω..., H) + q̃(ω...)*υ(ω..., T))
        else
            error("out of domain parameter")
        end
    end
end;

Let's check the performance of the naive implementation computing $V_0$ $L = 500$ times:

In [14]:
L = 500
@time for j = 1:L
    V₀ = V()
end
V₀

  5.983512 seconds (46.92 M allocations: 1.079 GiB, 1.86% gc time)


9.375000000000002

Now lets check the improved implementation $\upsilon(\omega_1 \omega_2 \ldots \omega_i)$

In [15]:
# redefine V with the improved implementation υ
V(ω::Toss...) = υ(ω...)
@time for j = 1:L
    # reset the dictionary for a fair comparison
    D = Dict()
    V₀ = V()
end
V₀

  0.467227 seconds (639.55 k allocations: 33.050 MiB, 2.41% gc time)


9.375000000000002